In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
# importing dataframes
districts = pd.read_csv('../input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv')
products = pd.read_csv('../input/learnplatform-covid19-impact-on-digital-learning/products_info.csv')

In [ ]:
districts.head(20)

In [ ]:
districts.tail


In [ ]:
products.head(20)

The engagement data are aggregated at school district level, and each file in the folder engagement_data represents data from one school district. The 4-digit file name represents district_id which can be used to link to district information in district_info.csv. The lp_id can be used to link to product information in product_info.csv.

In [ ]:
#first, we need to rename the column title "LP ID" to lp_id to merge both the products and engagement data in a single dataset 
products.rename(columns = {'LP ID': 'lp_id'}, inplace = True)
products.head()



In [ ]:
import glob 
path = '../input/learnplatform-covid19-impact-on-digital-learning/engagement_data'
engagement_files = glob.glob(os.path.join(path, "*.csv"))
my_list = []
for i in engagement_files:
    my_file = pd.read_csv(i)
    filename = os.path.splitext(i)
    my_file['district_id'] = os.path.basename(filename[0])
    my_list.append(my_file)
    engagement_data = pd.concat(my_list)

engagement_data

In [ ]:
print(districts.isnull().sum(axis=0))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
#counting the number of districts per states
plt.figure(figsize=(10,12))
sns.countplot(y ='state',data = districts,order=districts['state'].value_counts().index)
plt.show()

In [ ]:
#using a list of the USA states abbreviations
#list available here: https://gist.github.com/tlancon/9794920a0c3a9990279de704f936050c  Kudos to tlancon

us_states = {
	'Alabama': 'AL',
	'Alaska': 'AK',
	'Arizona': 'AZ',
	'Arkansas': 'AR',
	'California': 'CA',
	'Colorado': 'CO',
	'Connecticut': 'CT',
	'Delaware': 'DE',
	'District of Columbia': 'DC',
	'Florida': 'FL',
	'Georgia': 'GA',
	'Hawaii': 'HI',
	'Idaho': 'ID',
	'Illinois': 'IL',
	'Indiana': 'IN',
	'Iowa': 'IA',
	'Kansas': 'KS',
	'Kentucky': 'KY',
	'Louisiana': 'LA',
	'Maine': 'ME',
	'Maryland': 'MD',
	'Massachusetts': 'MA',
	'Michigan': 'MI',
	'Minnesota': 'MN',
	'Mississippi': 'MS',
	'Missouri': 'MO',
	'Montana': 'MT',
	'Nebraska': 'NE',
	'Nevada': 'NV',
	'New Hampshire': 'NH',
	'New Jersey': 'NJ',
	'New Mexico': 'NM',
	'New York': 'NY',
	'North Carolina': 'NC',
	'North Dakota': 'ND',
	'Ohio': 'OH',
	'Oklahoma': 'OK',
	'Oregon': 'OR',
	'Pennsylvania': 'PA',
	'Rhode Island': 'RI',
	'South Carolina': 'SC',
	'South Dakota': 'SD',
	'Tennessee': 'TN',
	'Texas': 'TX',
	'Utah': 'UT',
	'Vermont': 'VT',
	'Virginia': 'VA',
	'Washington': 'WA',
	'West Virginia': 'WV',
	'Wisconsin': 'WI',
	'Wyoming': 'WY'
}

In [ ]:
#replacing US state names by 2 letters state names
for i in districts.index:
    for key in us_states:
        if districts['state'][i] == key:
            districts.at[i,'state']=us_states[key]

In [ ]:
districts

In [ ]:
districts.dropna(inplace=True)
districts.reset_index(drop=True, inplace=True)
districts

In [ ]:
#separating black and hispanic data into two separate columns 

black = []
hispanic = []
for i in districts['pct_black/hispanic']:
    black.append(float(i.replace("[","").replace(']','').split(", ")[0]))
    hispanic.append(float(i.replace("[","").replace(']','').split(", ")[1]))
districts['black'] = black
districts['hispanic'] = hispanic
    


In [ ]:
districts

In [ ]:
black_hispanic = districts[['state','black','hispanic']]
black_hispanic

#We need to average the percentage of black and hispanic people for the different states are there are multiple occurences of certain states.

In [ ]:
#creation of a new dataframe filled with NaN

new_dataframe = pd.DataFrame(columns=['state', 'black', 'hispanic'],index=range(100))
new_dataframe

In [ ]:
new_dataframe

In [ ]:
#for each state, we insert the percentages of hispanic and black people to "newdataframe". If the state has already been inserted, then the values are averaged.

import numpy as np
count1 = 0
for i in black_hispanic['state']:
    count = 0
    while count1 < 87:
        if str(black_hispanic['state'].loc[count1]) == str(new_dataframe['state'].loc[count]):
            new_dataframe.at[count,'black'] = (black_hispanic['black'].loc[count1] + new_dataframe['black'].loc[count])/2
            new_dataframe.at[count,'hispanic'] = (black_hispanic['hispanic'].loc[count1] + new_dataframe['hispanic'].loc[count])/2
            count = 0
            count1 = count1+1
        if pd.isna(new_dataframe['state'].loc[count]) == True:
            new_dataframe.at[count,'state'] = black_hispanic['state'].loc[count1]
            new_dataframe.at[count,'black'] = black_hispanic['black'].loc[count1]
            new_dataframe.at[count,'hispanic'] = black_hispanic['hispanic'].loc[count1]
            count = 0
            count1 = count1 + 1
        if str(black_hispanic['state'].loc[count1]) != str(new_dataframe['state'].loc[count]):
            count = count +1
            
   
       

In [ ]:

new_dataframe =  new_dataframe.dropna()
new_dataframe

In [ ]:
#final list of states

for i in new_dataframe['state']:
    print(i)

In [ ]:
#converting values to float

new_dataframe[['black','hispanic']]=new_dataframe[['black','hispanic']].apply(pd.to_numeric) 

In [ ]:
#values are now converted

new_dataframe['black'].dtype

In [ ]:
#plotting the averaged percentage of black people per state on a US map.

import plotly.express as px

fig = px.scatter_geo(
    new_dataframe, locations="state", locationmode='USA-states',
    scope="usa",
    color='black', 
    size='black', hover_name="state", 
    range_color= [0, 1], 
    projection="albers usa", 
    title='Percentage of black people per US State', 
    color_continuous_scale = "Blues"
)

fig.show()

In [ ]:
#plotting the averaged percentage of hispanic people per state on a US map.

import plotly.express as px

fig = px.scatter_geo(
    new_dataframe, locations="state", locationmode='USA-states',
    scope="usa",
    color='hispanic', 
    size='hispanic', hover_name="state", 
    range_color= [0, 1], 
    projection="albers usa", 
    title='Percentage of hispanic people per US State', 
    color_continuous_scale = "Blues"
)

fig.show()

In [ ]:
#joining the datasets to include engagement data

engagement_data["district_id"] = engagement_data["district_id"].astype(str).astype(int)
districts_engagement = pd.merge(districts, engagement_data, left_on='district_id', right_on='district_id')
districts_engagement.head(20)

In [ ]:
#grouping by state and calculating the averaged engagement

avg_engagement_df = pd.DataFrame()
avg_engagement = districts_engagement.groupby('state')['engagement_index'].mean().reset_index(name='avg_engagement') #averaged engagement per state
#avg_engagement_df['avg_engagement'] = avg_engagement

In [ ]:
avg_engagement

In [ ]:
#plotting the averaged engagement per state

import plotly.express as px

fig = px.scatter_geo(
    avg_engagement, locations="state", locationmode='USA-states',
    scope="usa",
    color='avg_engagement', 
    size='avg_engagement', hover_name="state", 
    range_color= [0, 1], 
    projection="albers usa", 
    title='Averaged engagement per US State', 
    color_continuous_scale = "Blues"
)

fig.show()

In [ ]:
new_dataframe

In [ ]:
#inserting the engagement data to new_dataframe containing the averaged percentage of black and hispanic people per state
for i in new_dataframe['state']:
    for j in avg_engagement['state']:
        if i == j:
            new_dataframe['avg_engagement']=avg_engagement['avg_engagement']

In [ ]:
#resulting dataset 

new_dataframe

In [ ]:
#plotting engagement per state 

import matplotlib.pyplot as plt

ax = plt.subplot(111)


x = new_dataframe['state']
y = new_dataframe['avg_engagement']

ax.bar(x,y,color='r')


ax.autoscale(tight=True)

In [ ]:
#plotting percentage of black people per state

import matplotlib.pyplot as plt

ax = plt.subplot(111)

y = new_dataframe['black']
x = new_dataframe['state']


ax.bar(x,y,color='b')

ax.autoscale(tight=True)

#it looks like the states having a higher percentage of black people are having less engagement.

In [ ]:
#correlation between averaged percentage of black people and averaged engagement

new_dataframe[['avg_engagement','black']].corr(method='pearson') #looks like black people are less likely to engage

In [ ]:
#plotting the percentage of hispanic people per state

import matplotlib.pyplot as plt

ax = plt.subplot(111)

y = new_dataframe['hispanic']
x = new_dataframe['state']


ax.bar(x,y,color='g')

ax.autoscale(tight=True)

#The same applies to the states having a higher percentage of hispanic people.

In [ ]:
#correlation between averaged percentage of hispanic people and averaged engagement

new_dataframe[['avg_engagement','hispanic']].corr(method='pearson') #looks like hispanic people are less likely to engage

In [ ]:
#contrast between averaged percentage of black and hispanic people and the averaged engagement

import plotly.express as px
fig = px.scatter_geo(
    new_dataframe, locations="state", locationmode='USA-states',
    scope="usa",
    color='avg_engagement', 
    size='avg_engagement', hover_name="state", 
    range_color= [0, 1], 
    projection="albers usa", 
    title='Averaged engagement per US State', 
    color_continuous_scale = "Blues"
)

fig2 = px.scatter_geo(
    new_dataframe, locations="state", locationmode='USA-states',
    scope="usa",
    color='black', 
    size='black', hover_name="state", 
    range_color= [0, 1], 
    projection="albers usa", 
    title='Averaged percentage of black people per US State', 
    color_continuous_scale = "Blues"
)

fig3 = px.scatter_geo(
    new_dataframe, locations="state", locationmode='USA-states',
    scope="usa",
    color='hispanic', 
    size='hispanic', hover_name="state", 
    range_color= [0, 1], 
    projection="albers usa", 
    title='Averaged percentage of hispanic people per US State', 
    color_continuous_scale = "Blues"
)
fig.show()
fig2.show()
fig3.show()